In [1]:
# For dealing with files
import os
import shutil
from pathlib import Path
# For saving data matrix
import numpy as np
import random

# For normalising data
from statistics import pstdev,mean
# For using regex expressions
import re

In [4]:
# save where you currently are (for coming back afterwards)
#old_path = path('C:\Users\bjorn\Documents\Repositories\deep-machine-learning\Project')
old_path = os.getcwd()
# cd to sequencec directory
os.chdir('sequences')
'''# Get a list of all filenames inside
files = os.listdir()
print(files[0:10])
# Filter list using regex expressions
r_testimg_inputdata = re.compile('testimg.*.inputdata.txt')
r_trainimg_inputdata = re.compile('trainimg.*.inputdata.txt')
print(type(r_testimg_inputdata))
#all_train_filenames = list(filter(r_trainimg_inputdata.match, files))
#all_test_filenames = list(filter(r_testimg_inputdata.match, files))'''

# Fulhax galore pga pallar inte mer
prefix_train='trainimg-'
prefix_test='testimg-'
suffix='-inputdata.txt'
all_train_filenames = [prefix_train+str(i)+suffix for i in range(60000)]
all_test_filenames = [prefix_test+str(i)+suffix for i in range(10000)]



def preproces_data(filenames,absolute_coord=True,max_sequence_length=None):
    if max_sequence_length==None:
        max_sequence_length=117
    # pre-allocate data matrix
    nbrSequences=len(filenames)
    nbr_features = 4      
    data_tmp = np.empty((nbrSequences,max_sequence_length,nbr_features),dtype=np.int8)
    hist_data = np.zeros(nbrSequences)
    
    # Counter to keep track of number of sequences shorter than 61
    count_seq=0
    
    for sequnce_ix, file in enumerate(filenames):
        list_of_sequence=open(file, 'r').readlines()
        length_sequence = len(list_of_sequence)
        if length_sequence<=max_sequence_length:
            # How large padding is needed?
            padding = max_sequence_length-length_sequence
            count_seq+=1
            hist_data[sequnce_ix]=length_sequence
            if sequnce_ix%1000==0:
                print(sequnce_ix)
            tmp=np.empty((length_sequence,nbr_features))
            for time_ix,row in enumerate(list_of_sequence):
                (dx,dy,eos,eod) = [int(i) for i in row.split(' ')]
                tmp[time_ix]=[dx,dy,eos,eod]
            if absolute_coord:
                tmp=np.cumsum(tmp,axis=0) # Transform to absolute coordinates
                tmp_mean_x = mean(tmp[:,0])
                tmp_mean_y = mean(tmp[:,1])
                tmp[:,0] -= tmp_mean_x
                tmp[:,1] -= tmp_mean_y
                if padding>0:
                    #Pick 'padding' number of random integers between 0 (inclusive) and length of sequence (exclusive)
                    sample_range=range(length_sequence-1)
                    if padding>length_sequence:
                        rand_ix_tmp = random.sample(sample_range,length_sequence-1)
                        randy = random.sample(sample_range,padding-length_sequence)
                        print(padding,len(rand_ix_tmp)+len(randy))
                        #print(length_sequence,rand_ix)
                    random_ix = np.random.randint(0, high=length_sequence-1, size=padding)
                    random_ix=np.sort(random_ix)[::-1]
                    #print(random_ix)
                    midpoints_to_insert = np.empty((len(random_ix),4))
                    for counter_ix,padd_ix in enumerate(random_ix): # Loops in revere order to not interfer with later inserts
                        (x1,x2) = tmp[padd_ix:padd_ix+2,0]
                        (y1,y2) = tmp[padd_ix:padd_ix+2,1]
                        midpoints_to_insert[counter_ix,:] = [(x1 + x2)/2,(y1 + y2)/2,0,0]    
                    tmp=np.insert(tmp,random_ix+1,midpoints_to_insert, axis=0)
                    
                
            else:
                # Set start to origin for all numbers
                tmp[0,0] = 0
                tmp[0,1] = 0
            data_tmp[sequnce_ix,-max_sequence_length:,:]=tmp.copy()
    print(count_seq)
    data = data_tmp[:count_seq,:,:]
    print('Datan',data)
    print('HIST Datan',hist_data)
    return data,hist_data


test_data_abs_coord, test_hist_data = preproces_data(all_test_filenames,absolute_coord=True,max_sequence_length=60)
train_data_abs_coord, train_hist_data = preproces_data(all_train_filenames,absolute_coord=True,max_sequence_length=60)
test_data_rel_coord, test_hist_data = preproces_data(all_test_filenames,absolute_coord=False,max_sequence_length=60)
train_data_rel_coord, train_hist_data = preproces_data(all_train_filenames,absolute_coord=False,max_sequence_length=60)
os.chdir(old_path)

0


ValueError: Sample larger than population or is negative

In [ ]:
import matplotlib.pyplot as plt
print(test_data_rel_coord.shape)
# An "interface" to matplotlib.axes.Axes.hist() method
plt.subplot(1,2,1)
n, bins, patches = plt.hist(x=test_hist_data, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('test_hist_data')
plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
plt.subplot(1,2,2)
n, bins, patches = plt.hist(x=train_hist_data, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('train_hist_data')
plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
plt.show()

In [ ]:
np.save('test_data_rel_coord_60.npy',test_data_rel_coord)
np.save('train_data_rel_coord_60.npy',train_data_rel_coord)
#np.save('test_data_abs_coord.npy',test_data_abs_coord)
#np.save('train_data_abs_coord.npy',train_data_abs_coord)

In [3]:
os.chdir(old_path)